In [ ]:
from fedai.core import *

In [ ]:
import os
import numpy as np
from datetime import datetime
from fedai.servers import *
from fedai.clients import *
from fedai.utils import *
from omegaconf import DictConfig, OmegaConf
import hydra
import argparse


#### Hydra as our config manager

In [ ]:
path = 'folder/config/config.yaml'
config_path = os.path.dirname(path)
config_name = os.path.basename(path)
config_name = os.path.splitext(config_name)[0]

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')

fname = parser.parse_args().fname
config_path = os.path.dirname(fname)
config_name = os.path.splitext(os.path.basename(config_path))[0]

In [ ]:
@hydra.main(version_base=None, config_path=config_path, config_name=config_name)
def app(cfg: DictConfig) -> None:
    print(OmegaConf.to_yaml(cfg))
    loss_ds, gener_ds = load_ds(cfg)
    list_train_ds, list_eval_ds, tokenizer, datacollator = loss_ds
    list_train_ds_genr, list_eval_ds_genr = gener_ds
    

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/ahmed/.local/share/jupyter/runtime/kernel-v345b718cf6313858386357bb42dd52bcafd1180c9.json


SystemExit: 2

/home/ahmed/miniconda3/envs/fedai/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Test main


In [ ]:
from fedai.core import *

In [ ]:
from fedai.data import get_dolly
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
dolly = get_dolly('', tokenizer)

In [ ]:
(lst_train_ds, lst_eval_set, tokenizer, data_collator), (lst_train_ds_genr, lst_eval_set_genr)  = dolly

In [ ]:
lst_data_dict = [{'train': tr, 'test': test} for tr, test in zip(lst_train_ds, lst_eval_set)]
lst_data_dict_genr = [{'train': tr, 'test': test} for tr, test in zip(lst_train_ds_genr, lst_eval_set_genr)]

In [ ]:
len(lst_data_dict) == len(lst_data_dict_genr)

True

In [ ]:
def criterion(y_pred, y_true= None):
    loss = y_pred.loss
    return loss

In [ ]:
kwargs = {'lst_gen_data_dict': lst_data_dict_genr,
          'tokenizer': tokenizer,
          'collat_fn': data_collator,
          'criterion': criterion
          }

In [ ]:
from fedai.core import get_cfg
cfg = get_cfg()
cfg

{'dataset': 'dolly', 'model': 'openai-community/gpt2', 'max_length': 1022, 'num_clients': 80, 'iid': 'dir0.5', 'batch_size': 1, 'dataset_subsample': 1.0, 'eval_metric': 'loss', 'log_root': 'logs', 'save_dir': 'checkpoints', 'lora_alpha': 8, 'lora_dropout': 0.05, 'r': 32, 'target_modules': ['c_attn'], 'lr': 5e-05, 'optimizer': 'Adam', 'name': 'mira', 'device': 0, 'bias_sampling': False, 'num_clients_per_task': 10, 'use_prompts': True, 'rounds': 40, 'm': 0.05}

In [ ]:
cfg.log_dir = os.path.join(cfg.log_root, 
                            datetime.now().strftime("%Y%m%d_%H%M%S"))
        
cfg.save_dir = os.path.join(cfg.save_dir, 
                            datetime.now().strftime("%Y%m%d_%H%M%S"))

In [ ]:
len(lst_data_dict)

80

In [ ]:
server = get_server(cfg, lst_data_dict, model= None, holdout_ds= None, **kwargs)

/home/ahmed/miniconda3/envs/fedai/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
server.client_list

<fedai.utils.LazyInit>

In [ ]:
server.client_list = LazyList(server, Client_mira)


In [ ]:
client_indices_rounds = get_client_indices_rounds(cfg)

In [ ]:
len(client_indices_rounds)

40

In [ ]:
# for t in range(1, 41):
#     clients_history = []
#     local_ds_len = {}
#     prev_clients = set()    
#     for client in server.get_selected_client(client_indices_rounds[t-1]):
#         client.model = server.send_model(client)
#         client.init_local_train("self.output_dir")
#         client.optimizer = get_class('torch.optim', server.cfg.optimizer)(client.model.parameters(), lr= server.cfg.lr)
#         history = [0]# training happens at this step
#         client.model, local_ds_len, prev_clients, _ = client.terminate_local_train(t, local_ds_len, prev_clients)
#         clients_history.append(history)
#         print(f"client {client.idx} finished round {t}")
#         del client#server.save_space(client)
#         server.client_list.clear_cache()
#         import gc
#         gc.collect()

client 31 finished round 1
client 71 finished round 1
client 44 finished round 1
client 20 finished round 1
client 28 finished round 2
client 37 finished round 2
client 24 finished round 2
client 30 finished round 2
client 73 finished round 3
client 58 finished round 3
client 29 finished round 3
client 1 finished round 3
client 8 finished round 4
client 76 finished round 4
client 57 finished round 4
client 47 finished round 4
client 28 finished round 5
client 39 finished round 5
client 37 finished round 5
client 63 finished round 5
client 65 finished round 6
client 70 finished round 6
client 8 finished round 6
client 16 finished round 6
client 51 finished round 7
client 32 finished round 7
client 38 finished round 7
client 47 finished round 7
client 30 finished round 8
client 37 finished round 8
client 51 finished round 8
client 78 finished round 8
client 38 finished round 9
client 51 finished round 9
client 18 finished round 9
client 57 finished round 9
client 29 finished round 10
cli